In [34]:
import requests
from bs4 import BeautifulSoup
import csv
import xml.etree.ElementTree as ET
import time
from queue import Queue
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [36]:
df = pd.read_csv("university_data3.csv")

In [39]:
df.isna().sum()

URL            0
Heading      446
Content    19019
dtype: int64

In [42]:
df["Content"] = df["Content"].fillna("No Content")

In [43]:
df

,URL,Heading,Content
0,https://www.hull.ac.uk/,Come to an Open Day this January,No Content
1,https://www.hull.ac.uk/,This. Is. Hull.,And it's yours for the taking.
2,https://www.hull.ac.uk/,The gold standard for teaching,No Content
3,https://www.hull.ac.uk/,Find your extraordinary,No Content
4,https://www.hull.ac.uk/,Study with us,"From our friendly academics, to our beautiful ..."
...,...,...,...
42182,https://www.hull.ac.uk/about.aspx,A history set in stone,The University of Hull was founded in 1927 – o...
42183,https://www.hull.ac.uk/about.aspx,Where are we heading next?,Our Royal Charter says our mission is to “…adv...
42184,https://www.hull.ac.uk/about.aspx,A serious investment in your future,We’re committed to providing our students with...
42185,https://www.hull.ac.uk/about.aspx,At the cutting edge of research,The University has particular research experti...


In [44]:
df.isna().sum()

URL          0
Heading    446
Content      0
dtype: int64

In [45]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import csv
import requests
import xml.etree.ElementTree as ET

def get_sitemap_urls(sitemap_url):
    response = requests.get(sitemap_url)
    sitemap = response.content
    root = ET.fromstring(sitemap)
    urls = [url.text for url in root.findall('.//{http://www.sitemaps.org/schemas/sitemap/0.9}loc')]
    return urls

def scrape_page(url, writer):
    try:
        print(f"Scraping URL: {url}")  # Print the URL being scraped
        driver.get(url)
        
        # Wait for the page to load dynamically
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Extracting the page title
        page_title = soup.find('title').get_text(strip=True) if soup.find('title') else 'No Title'

        # Extracting headings and their associated content
        headings = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
        for heading in headings:
            text_content = heading.get_text(' ', strip=True)
            following_elements = heading.find_next_siblings()
            content = ' '.join(element.get_text(' ', strip=True) for element in following_elements if element.name in ['p', 'ul', 'ol'])
            writer.writerow([url, page_title, text_content, content])
    except Exception as e:
        print(f"Error scraping {url}: {e}")

# Selenium WebDriver setup
options = Options()
options.headless = True
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

# Get URLs from sitemap
sitemap_url = 'https://www.hull.ac.uk/sitemap.xml'
urls = get_sitemap_urls(sitemap_url)

# Scrape and save data to CSV
output_file = 'university_data_updated.csv'
with open(output_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['URL', 'Page Title', 'Heading', 'Content'])
    
    for url in urls:
        scrape_page(url, writer)

driver.quit()


Scraping URL: https://www.hull.ac.uk/
Scraping URL: https://www.hull.ac.uk/index.aspx
Scraping URL: https://www.hull.ac.uk/search/
Scraping URL: https://www.hull.ac.uk/study/webinars.aspx
Scraping URL: https://www.hull.ac.uk/study/undergraduate/
Scraping URL: https://www.hull.ac.uk/study/undergraduate/diagnostic-radiography-bsc-hons.aspx
Scraping URL: https://www.hull.ac.uk/study/undergraduate/accounting-and-financial-management-bsc-hons.aspx
Scraping URL: https://www.hull.ac.uk/study/undergraduate/accounting-bsc-hons.aspx
Scraping URL: https://www.hull.ac.uk/study/undergraduate/acting-ba-hons.aspx
Scraping URL: https://www.hull.ac.uk/study/undergraduate/biochemistry-bsc-hons.aspx
Scraping URL: https://www.hull.ac.uk/study/undergraduate/biology-bsc-hons.aspx
Scraping URL: https://www.hull.ac.uk/study/undergraduate/biomedical-engineering-beng-meng.aspx
Scraping URL: https://www.hull.ac.uk/study/undergraduate/biomedical-science-bsc-hons.aspx
Scraping URL: https://www.hull.ac.uk/study/und